In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_Columns', 100)

In [2]:
cartlog = pd.read_feather('../inputs/cartlog.f')
product_master = pd.read_feather('../inputs/product_master.f')
meta = pd.read_feather('../inputs/meta.f')
user_master = pd.read_feather('../inputs/user_master.f')
test = pd.read_csv('../inputs/test.csv')
display_action_id = pd.read_csv('../inputs/display_action_id.csv')

product_master['JAN'] = product_master['JAN'].astype(str)

In [3]:
test_sessions = test["session_id"].unique()
print(len(test_sessions))
test_input_log = cartlog[cartlog["session_id"].isin(test_sessions)]

56486


In [4]:
target_category = [
    38,  # アイスクリーム__ノベルティー
    110,  # スナック・キャンディー__ガム
    113,  # スナック・キャンディー__シリアル
    114,  # スナック・キャンディー__スナック
    134,  # チョコ・ビスクラ__チョコレート
    171,  # ビール系__RTD
    172,  # ビール系__ノンアルコール
    173,  # ビール系__ビール系
    376,  # 和菓子__米菓
    435,  # 大型PET__無糖茶（大型PET）
    467,  # 小型PET__コーヒー（小型PET）
    537,  # 水・炭酸水__大型PET（炭酸水）
    539,  # 水・炭酸水__小型PET（炭酸水）
    629,  # 缶飲料__コーヒー（缶）
    768,  # 麺類__カップ麺
]

In [5]:
class RetailDataset:
    def __init__(self, thres_sec, cartlog, product, meta):
        self.thres_sec = thres_sec
        self.cartlog = cartlog
        self.product_master = product
        self.meta = meta.copy()
        
        meta['time_elapsed_sec'] = meta['time_elapsed'] * 60
        self.meta.loc[self.meta['time_elapsed_sec'].isnull(), 'time_elapsed_sec'] = thres_sec
        
    def get_log_first_half(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        """
        first_half_sessions = set(
            self.meta.query("date < '2020-08-01'")["session_id"].unique()
        )
        return self.cartlog[self.cartlog["session_id"].isin(first_half_sessions)]

    def get_train_output_log(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        """
        return pd.merge(
            self.get_log_first_half(),
            self.meta[["session_id", "time_elapsed_sec"]],
            on=["session_id"],
            how="inner",
        ).query("spend_time > time_elapsed_sec")

    def get_train_sessions(self) -> set:
        """以下の条件を満たすセッションを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        """
        return set(self.get_train_output_log()["session_id"].unique())

    def get_train_input_log(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        - 指定した時間(thres_sec)より前のログである
        """
        train_sessions = self.get_train_sessions()
        return pd.merge(
            self.get_log_first_half()[
                self.get_log_first_half()["session_id"].isin(train_sessions)
            ],
            self.meta[["session_id", "time_elapsed_sec"]],
            on=["session_id"],
            how="inner",
        ).query("spend_time <= time_elapsed_sec")

    def get_payment_sessions(self) -> set:
        """以下の条件を満たすセッションを取得する
        - 決済を行った
        """
        return set(self.cartlog.query("is_payment == 1")["session_id"].unique())

    def agg_payment(self, cartlog) -> pd.DataFrame:
        """セッションごと・商品ごとの購買個数を集計する"""
        # 購買情報は商品のものだけ.
        target_index = cartlog["kind_1"] == "商品"

        # JANコード (vale_1)ごとに商品の購入個数(n_items)を足し算
        agg = (
            cartlog.loc[target_index]
            .groupby(["session_id", "value_1"])["n_items"]
            .sum()
            .reset_index()
        )
        agg = agg.rename(columns={"value_1": "JAN"})
#         agg = agg.astype({"JAN": int})
        return agg

    def get_train_target(self) -> pd.DataFrame:
        """学習で使用するセッションの目的変数を取得する"""
        # 空のターゲット用データフレームを用意する
        train_sessions = self.get_train_sessions()
        train_target = pd.DataFrame(
            np.zeros((len(train_sessions), len(target_category))),
            index=train_sessions,
            columns=target_category,
        ).astype(int)
        train_target.index.name = "session_id"

        # 集計する
        train_output_log = self.get_train_output_log()
        train_items_per_session_jan = self.agg_payment(train_output_log)
        train_items_per_session_target_jan = pd.merge(
            train_items_per_session_jan,
            self.product_master[["JAN", "category_id"]],
            on="JAN",
            how="inner",
        ).query("category_id in @target_category")
        train_target_pos = (
            train_items_per_session_target_jan.groupby(["session_id", "category_id"])[
                "n_items"
            ]
            .sum()
            .unstack()
            .fillna(0)
            .astype(int)
        )
        train_target_pos[train_target_pos > 0] = 1
        train_target_pos[train_target_pos <= 0] = 0

        train_target.loc[train_target_pos.index] = train_target_pos.values
        return train_target[target_category].reset_index()

In [6]:
def get_train_log(elapsed_min):
    dataset = RetailDataset(elapsed_min*60, cartlog, product_master, meta)
    train_input_log = dataset.get_train_input_log()
    y_train = dataset.get_train_target()
    print('train_session', y_train.shape)
    return train_input_log, y_train

In [7]:
ELAPSED_MIN = [0, 3, 5, 10]

In [8]:
def save_train_log():
    for elap_min in ELAPSED_MIN:
        train_log, train_y = get_train_log(elap_min)
        train_log = train_log.reset_index(drop=True)
        train_y = train_y.reset_index(drop=True)
        train_y.columns = [str(c) for c in train_y.columns]
        train_log.to_feather('../inputs/train_log_{}.f'.format(elap_min))
        train_y.to_feather('../inputs/train_y_{}.f'.format(elap_min))

In [9]:
def load_train_log():
    train_log = {}
    train_y = {}
    for elap_min in ELAPSED_MIN:
        log = pd.read_feather('../inputs/train_log_{}.f'.format(elap_min))
        y = pd.read_feather('../inputs/train_y_{}.f'.format(elap_min))
        train_log[elap_min] = log
        train_y[elap_min] = y
    return train_log, train_y

In [10]:
# save_train_log()

### ユーザ情報

In [11]:
user_features = pd.merge(meta[["session_id", "user_id"]], user_master, on="user_id", how="left").drop(columns=["user_id"])
user_features.loc[user_features['age'] >= 80, 'age'] = np.NaN
user_features.loc[user_features['age'] < 10, 'age'] = np.NaN
user_features.loc[user_features['gender'] > 1, 'gender'] = np.NaN

### メタ情報

In [12]:
meta_features = meta.copy()
meta_features['year'] = meta_features['date'].dt.year
meta_features['month'] = meta_features['date'].dt.month
meta_features['day'] = meta_features['date'].dt.day
meta_features['dow'] = meta_features['date'].dt.dayofweek
meta_features['doy'] = meta_features['date'].dt.dayofyear
le = preprocessing.LabelEncoder()
meta_features['userid'] = le.fit_transform(meta_features['user_id'])
meta_features.drop(columns=['user_id', 'date', 'time_elapsed', 'date_str'], inplace=True)

### ディスプレイアクション

In [13]:
disp_name_dic = {}
for i, disp in enumerate(display_action_id['display_name'].unique()):
    disp_name_dic[disp] = 'disp_cnt_{}'.format(i)
    
act_name_dic = {}
for i, action in enumerate(display_action_id['action_name'].unique()):
    act_name_dic[action] = 'act_cnt_{}'.format(i)

In [14]:
def get_display_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    disp_group_count = merge.groupby(['session_id', 'display_name']).size().reset_index().rename(columns={0:'disp_name_count'})
    disp_name_pivot = disp_group_count.pivot_table(index='session_id', columns='display_name', values='disp_name_count', aggfunc='sum')
    disp_name_pivot = disp_name_pivot.reset_index().fillna(0).rename(columns=disp_name_dic)
    
    disp_out = disp_name_pivot[['session_id']].copy()
    for val in disp_name_dic.values():
        disp_out[val] = 0
    
    for col in disp_name_pivot.columns:
        if col == 'session_id':
            continue
        disp_out[col] = disp_name_pivot[col]
    
    return disp_out

In [15]:
def get_action_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    act_group_count = merge.groupby(['session_id', 'action_name']).size().reset_index().rename(columns={0:'act_name_count'})
    act_name_pivot = act_group_count.pivot_table(index='session_id', columns='action_name', values='act_name_count', aggfunc='sum')
    act_name_pivot = act_name_pivot.reset_index().fillna(0).rename(columns=act_name_dic)
    
    act_out = act_name_pivot[['session_id']].copy()
    for val in act_name_dic.values():
        act_out[val] = 0
    
    for col in act_name_pivot.columns:
        if col == 'session_id':
            continue
        act_out[col] = act_name_pivot[col]
    
    return act_out

### セッション単位の特徴量

In [16]:
def get_session_kind_group(input_log):
    kind_name ={
        'クーポン': 'coupon',
        '会計': 'kaikei',
        'キー': 'key',
        'カテゴリ': 'categry',
        'バーコードスキャン': 'barcode',
        'UUID': 'uuid',
        '使用ポイント': 'usedpoint',
        '確認': 'confirm',
        'ブランドスイッチ': 'bland',
        'レシピ': 'recipe',
        'スマホスキャン': 'smartphone',
        '磁気スキャン': 'magnetic',
        'レコメンド': 'recommend',
        '倍率ポイント': 'point',
    }
    group_count = input_log[input_log['kind_1'] == '商品'].groupby(["session_id"]).size().rename('group_count_'+'item')
    for kind, name in kind_name.items():
        tmp = input_log[input_log['kind_1'] == kind].groupby(["session_id"]).size().rename('group_count_'+name)
        group_count = pd.concat([group_count, tmp], axis=1)
        
    return group_count.reset_index()

In [17]:
def get_session_info(input_log):
    # アクション数
    n_actions = input_log.groupby(["session_id"]).size().rename("n_actions")
    # 経過時間の平均
    mean_spend_time = input_log.groupby(["session_id"])["spend_time"].mean()
    
    session_features = pd.concat([
        n_actions,
        mean_spend_time,
    ], axis=1)
    return session_features.reset_index()

### セッション単位で集計

In [18]:
def get_session_features(input_log):
    df_ses = pd.DataFrame(columns=['session_id'])
    
    session_feat = [
        get_session_info(input_log),
        get_session_kind_group(input_log),
        get_display_name_feature(input_log),
        get_action_name_feature(input_log),
    ]
    
    for feat in session_feat:
        df_ses = pd.merge(df_ses, feat, on='session_id', how='outer')
        
    return df_ses

### 特徴量を集約する

In [19]:
def merge_features(input_log, session):
    feat_list = [
        get_session_features(input_log),
        user_features,
        meta_features,
    ]
    out = pd.DataFrame({"session_id": session})
    for feat in feat_list:
        out = pd.merge(out, feat, on="session_id", how="left")

    assert len(session) == len(out)
    return out

In [20]:
def get_train_all_features(elapsed_min, train_log_list, train_y_list):
    train_input_log = train_log_list[elapsed_min]
    y_train = train_y_list[elapsed_min]
    
    train_features = merge_features(train_input_log, y_train['session_id'])
    print('train_session', len(y_train))
    return train_features, y_train

In [21]:
def get_test_all_feature(elapsed_min):
    test_meta = meta[meta['session_id'].isin(test_sessions)]
    test_meta = test_meta[test_meta['time_elapsed'] == elapsed_min]
    test_input_elapsed = pd.merge(test_input_log, test_meta[['session_id']], on='session_id', how='left')
    
    test_features = merge_features(test_input_elapsed, test_meta['session_id'])
    print('test_session', len(test_meta))
    return test_features

In [22]:
n_fold = 4

In [23]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'seed' : 0,
    'learning_rate':  0.1,
    'max_depth': 6,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

In [24]:
def train_lgbm(X, y, params=lgbm_param):

    fold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
    cv = fold.split(X, y)
    
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    cat_feat = ['age', 'gender', 'dow']

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train, categorical_feature = cat_feat)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train, categorical_feature = cat_feat)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    categorical_feature = cat_feat,
                                                    num_boost_round=10000,
                                                    early_stopping_rounds=100,
                                                    verbose_eval=-1)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)

    score = roc_auc_score(y, oof_pred)
    print('--- FINISHED \ whole score: {:.4f} ---'.format(score))
    return oof_pred, models, score

In [25]:
def predict(models, feature):
    pred_list = []
    for i, model in enumerate(models):
        pred = model.predict(feature, num_iteration = model.best_iteration)
        pred_list.append(pred)
    
    score = np.mean(pred_list, axis=0)
    return score

In [26]:
train_log_list, train_y_list = load_train_log()

In [27]:
%%time
df_pred_all = pd.DataFrame()
df_score_all = pd.DataFrame(index=ELAPSED_MIN, columns=target_category).fillna(0.0)

for elapsed_min in ELAPSED_MIN:
    print(f'===== {elapsed_min} =====')
    train_features, y_train = get_train_all_features(elapsed_min, train_log_list, train_y_list)
    test_features = get_test_all_feature(elapsed_min)

    df_pred = pd.DataFrame(index=test_features['session_id'])
    train_features.drop(columns=['session_id'], inplace=True)
    test_features.drop(columns=['session_id'], inplace=True)
    
    models_list = []
    for target in y_train.columns:
        if target == 'session_id':
            continue
        
        print(f"---- id = {target} -----")
        oof, models, score = train_lgbm(train_features, y_train[target])
        models_list.append(models)

        pred = predict(models, test_features)
        df_pred[target] = pred
        df_score_all.loc[target, elapsed_min] = score
        
    df_pred_all = pd.concat([df_pred_all, df_pred])
    print(len(df_pred_all))

===== 0 =====
train_session 641293
test_session 14277
---- id = 38 -----
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1406]	valid_0's auc: 0.714898
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1484]	valid_0's auc: 0.716805
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1400]	valid_0's auc: 0.717708
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1483]	valid_0's auc: 0.717645
--- FINISHED \ whole score: 0.7168 ---
---- id = 110 -----
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[192]	valid_0's auc: 0.661706
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[502]	valid_0's auc: 0.664069
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[450]	valid_0's auc: 0.66372
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[248]	valid_0's auc: 0.670215
--- FINISHED \ whole score: 0.6643 ---
---- id = 113 -----
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[345]	valid_0's auc: 0.650995
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[583]	valid_0's auc: 0.655532
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[473]	valid_0's auc: 0.654009
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[354]	valid_0's auc: 0.659334
--- FINISHED \ whole score: 0.6550 ---
---- id = 629 -----
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[408]	valid_0's auc: 0.676849
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[216]	valid_0's auc: 0.691357
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[529]	valid_0's auc: 0.681421
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[293]	valid_0's auc: 0.670312
--- FINISHED \ whole score: 0.6782 ---
---- id = 768 -----
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[325]	valid_0's auc: 0.607987
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[201]	valid_0's auc: 0.609242
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[170]	valid_0's auc: 0.604784
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[407]	valid_0's auc: 0.603449
--- FINISHED \ whole score: 0.6062 ---
56486
CPU times: user 1d 16h 59min 24s, sys: 33min 45s, total: 1d 17h 33min 10s
Wall time: 1h 45min 45s


In [28]:
df_pred_all.head()

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
session_id,,,,,,,,,,,,,,,
663721,0.046360,0.017893,0.127007,0.309266,0.070504,0.054927,0.026186,0.089319,0.123792,0.022346,0.021288,0.037506,0.155968,0.091482,0.087610
663761,0.140151,0.019576,0.107046,0.362490,0.155115,0.206387,0.098059,0.085713,0.136940,0.061658,0.022267,0.234409,0.201832,0.010838,0.176517
663763,0.097266,0.021723,0.077895,0.297487,0.123363,0.160563,0.091828,0.085982,0.090588,0.088164,0.025448,0.026224,0.047783,0.009447,0.223343
663775,0.093764,0.015723,0.054708,0.238869,0.105243,0.092402,0.102250,0.112983,0.121714,0.075993,0.019150,0.072471,0.074027,0.009508,0.123662
663778,0.155856,0.066909,0.067096,0.250836,0.105997,0.060668,0.027367,0.054868,0.100314,0.049667,0.059055,0.068055,0.055271,0.059616,0.147264


In [29]:
assert len(df_pred_all) == len(test)

In [30]:
submission = pd.merge(test[['session_id']], df_pred_all.reset_index(), on='session_id', how='inner')
assert len(submission) == len(test)

In [31]:
submission.drop(columns='session_id').to_csv('../outputs/submission.csv', index=False)